In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from scipy.stats import uniform
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier

In [2]:
train = pd.read_csv('train.csv', header = 0, index_col= 0)
test = pd.read_csv('test.csv', header = 0, index_col= 0)
print(train.shape)
print(test.shape)

(260601, 39)
(86868, 38)


In [3]:
import warnings
warnings.filterwarnings("ignore")
# get dummy variables
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# create set of dependent variables and independent variables and scale.
X = train.drop(['damage_grade'], axis = 1)
y = train['damage_grade']

scaler = StandardScaler()
X = scaler.fit_transform(X)
test = scaler.fit_transform(test)

# split to test out for f1 score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 123)

# add column names to x_train and x_test for feature selection process
column_names = train.drop(['damage_grade'], axis = 1)
X_train = pd.DataFrame(X_train, columns = column_names.columns)
X_test = pd.DataFrame(X_test, columns = column_names.columns)
test = pd.DataFrame(test, columns = column_names.columns)

print(X_train.shape)
print(X_test.shape)
print(test.shape)

(174602, 68)
(85999, 68)
(86868, 68)


In [ ]:
# Based on the feature importance, new data will be tested
select_list = ['geo_level_1_id', 
                   'geo_level_2_id', 
                   'geo_level_3_id', 
                   'age', 
                   'area_percentage', 
                   'height_percentage', 
                   'count_families', 
                   'foundation_type_r', 
                   'count_floors_pre_eq', 
                   'has_superstructure_mud_mortar_stone',
                   'has_superstructure_timber',
                   'ground_floor_type_v',
                   'land_surface_condition_t',
                   'land_surface_condition_n',
                   'roof_type_n',
                   'other_floor_type_q']
X_train = X_train[select_list]
X_test = X_test[select_list]
test = test[select_list]

clf1 = LGBMClassifier(max_depth=7, learning_rate=.05)
clf2 = KNeighborsClassifier(n_neighbors=7)
clf3 = SVC(gamma='scale', kernel='rbf', probability=True)
eclf = VotingClassifier(estimators=[('lgbm', clf1), ('knn', clf2), ('svc', clf3)], voting='hard')

# clf1 = clf1.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds = 50, eval_metric = 'logloss')
# clf2 = clf2.fit(X_train, y_train)
# clf3 = clf3.fit(X_train, y_train)
eclf = eclf.fit(X_train, y_train)

# for clf, label in zip([clf1, clf2, clf3, eclf], ['LGBM', 'KNN', 'SVC', 'Ensemble']):
#     scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
#     print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

predictions = eclf.predict(test)
print(predictions)

result = pd.DataFrame()
result['building_id'] = btest['building_id']
result['damage_grade'] = prediction
result.to_csv('lgbm_prediction_result.csv',index = None)
print("预测结果已生成！")